In [ ]:
pip install pywin32


In [ ]:
pip install pyttsx3


In [ ]:
pip install SpeechRecognition


In [ ]:
pip install pyaudio

# Face & Object Detection Process

In [ ]:
# Only Detect faces
import cv2
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
cap=cv2.VideoCapture(0)
while True:
    ret,frame=cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,1.1,5) # detectMultiScale- scan and detect faces ,1.1 balance, not too slow,blind ,minNeighbors=5
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w , y+h),(0,255,0),2) # x=how far from left,y= how dar from top,w=width of face,h=height of face
    cv2.imshow("Webcam face Detection",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    

In [2]:
# Face,Smile,Eye Detection
import cv2
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade=cv2.CascadeClassifier("haarcascade_eye.xml")
smile_cascade=cv2.CascadeClassifier("haarcascade_smile.xml")
cap=cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,1.1,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w , y+h),(0,255,0),2)
    roi_gray=gray[y:y+h,x:x+w] # roi=reseon of intrest [150:150+80 , 100:100+80]
    roi_color=frame[y:y+h,x:x+w]
    eye=eye_cascade.detectMultiScale(roi_gray,1.1,10)
    if len(eye)>0:
        cv2.putText(frame,"Eye Detected",(x,y-40),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),1)
    smile=smile_cascade.detectMultiScale(roi_gray,1.7,20)
    if len(smile)>0:
        cv2.putText(frame,"Smile Detected",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),1)
    if len(faces)>0:
        cv2.putText(frame,"Face Detected",(x,y-20),cv2.FONT_HERSHEY_SIMPLEX,0.6,(0,255,0),1)
    cv2.imshow("Smart Face Detector",frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
    

In [1]:
# Hand Detection 
import cv2
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)
with mp_hands.Hands(model_complexity=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # 👉 Use flip(1) for mirror view without swapping left/right
        frame = cv2.flip(frame, 1)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        if result.multi_hand_landmarks:
            for hand_landmarks, hand_label in zip(result.multi_hand_landmarks, result.multi_handedness):
                # extract left/right from mp result
                label = hand_label.classification[0].label  # "Left" or "Right"
                # Draw hand landmarks
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                # Show label on screen
                cv2.putText(frame, label + " Hand",(10, 70), cv2.FONT_HERSHEY_SIMPLEX,1, (0, 255, 0), 1)
        cv2.imshow("Hand Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# Finger Counting Using Webcam
import cv2
import mediapipe as mp
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1,min_detection_confidence=0.7,min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils
tip_ids = [4, 8, 12, 16, 20]
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(imgRGB)
    if result.multi_hand_landmarks:
        handLms = result.multi_hand_landmarks[0]
        lm_list = []
        # Get hand type (Left/Right)
        hand_type = result.multi_handedness[0].classification[0].label
        for id, lm in enumerate(handLms.landmark):
            cx, cy = int(lm.x * w), int(lm.y * h)
            lm_list.append((cx, cy))
            fingers = []
        # --------------------- Thumb Logic ----------------------
        if hand_type == "Right":
            # For right hand (mirror flip)
            if lm_list[4][0] > lm_list[3][0]:
                fingers.append(1)
            else:
                fingers.append(0)

        else:  # Left Hand
            if lm_list[4][0] < lm_list[3][0]:
                fingers.append(1)
            else:
                fingers.append(0)
        # --------------------- Other 4 Fingers ----------------------
        for i in range(1, 5):
            if lm_list[tip_ids[i]][1] < lm_list[tip_ids[i] - 2][1]:
                fingers.append(1)
            else:
                fingers.append(0)

        total = fingers.count(1)
        # Draw count box
        cv2.rectangle(frame, (20, 20), (200, 150), (0, 255, 0), -1)
        cv2.putText(frame, f"{total}", (70, 120),cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 5)
        # Show hand type
        cv2.putText(frame, hand_type, (20, 180),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)
    cv2.imshow("Finger Counter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


# PC Controller

In [1]:
# Valume controlar by Hand
import cv2
import mediapipe as mp
import math
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL

# ---------------------- Volume Setup (OLD + WORKING METHOD) ------------------------
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]

# ---------------------- Mediapipe Hands ------------------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(imgRGB)

    if result.multi_hand_landmarks:
        for handLMs in result.multi_hand_landmarks:
            lm_list = []
            for id, lm in enumerate(handLMs.landmark):
                h, w, c = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lm_list.append((cx, cy))

            x1, y1 = lm_list[4]   # thumb tip
            x2, y2 = lm_list[8]   # index tip

            length = math.hypot(x2 - x1, y2 - y1)

            # Map distance to volume
            vol = int((length - 20) / (200 - 20) * (max_vol - min_vol) + min_vol)
            vol = max(min(vol, max_vol), min_vol)
            volume.SetMasterVolumeLevel(vol, None)

            cv2.circle(frame, (x1, y1), 10, (255, 0, 255), -1)
            cv2.circle(frame, (x2, y2), 10, (255, 0, 255), -1)
            cv2.line(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Show Volume %
            percentage = int((vol - min_vol) / (max_vol - min_vol) * 100)
            cv2.putText(frame, f"Volume: {percentage}%", (40, 70),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Volume Gesture Control", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
# Hand Gesture + Voice Command + Voice Typing (toggle) PC Controller with HUD
# Mixed Mode with extended gesture set (Drag, swipes, media, system gestures, etc.)
import cv2
import mediapipe as mp
import pyautogui
import time
import math
import threading
import speech_recognition as sr
import pyttsx3
import os
import webbrowser
import subprocess
import shutil
import ctypes

# -----------------------
# Configuration & setup
# -----------------------
pyautogui.FAILSAFE = False
SCREEN_W, SCREEN_H = pyautogui.size()

mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils
hands_detector = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.6, min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# HUD state
hud_text = ""
hud_icon = None
hud_timer = 0
HUD_SHOW_TIME = 1.2

# Gesture state / debouncing
dragging = False
drag_start = None
last_click_time = 0
voice_typing_enabled = False

# New gesture state variables
prev_palm = None
prev_time = time.time()
last_gesture_times = {}
gesture_cooldown = 0.8  # seconds between same gesture
shake_buffer = []
sequence_buffer = []
sequence_time = None

# TTS
tts_engine = pyttsx3.init()
def speak(text):
    try:
        tts_engine.say(text)
        tts_engine.runAndWait()
    except Exception:
        pass

# -----------------------
# HUD Helper
# -----------------------
def set_hud(text, icon=None):
    global hud_text, hud_icon, hud_timer
    hud_text = text
    hud_icon = icon
    hud_timer = time.time()

# -----------------------
# Utilities (apps / web / brightness / system)
# -----------------------
def safe_start_command(cmd_list, friendly_name=None):
    try:
        subprocess.Popen(cmd_list, shell=False)
        set_hud(f"Opened {friendly_name or cmd_list[0]}", "open")
        speak(f"Opening {friendly_name or cmd_list[0]}")
        return True
    except Exception:
        try:
            os.system(" ".join(cmd_list))
            set_hud(f"Opened {friendly_name or cmd_list[0]}", "open")
            speak(f"Opening {friendly_name or cmd_list[0]}")
            return True
        except Exception:
            set_hud(f"Cannot open {friendly_name or cmd_list[0]}", "error")
            speak(f"Failed to open {friendly_name or cmd_list[0]}")
            return False

def open_url(url, friendly_name=None):
    try:
        webbrowser.open(url, new=2)
        set_hud(f"Opened {friendly_name or url}", "web")
        speak(f"Opening {friendly_name or url}")
        return True
    except Exception:
        set_hud(f"Cannot open {friendly_name or url}", "error")
        speak(f"Failed to open {friendly_name or url}")
        return False

def open_windows_uri(uri, friendly_name=None):
    try:
        os.system(f"start {uri}")
        set_hud(f"Opened {friendly_name or uri}", "app")
        speak(f"Opening {friendly_name or uri}")
        return True
    except Exception:
        set_hud(f"Cannot open {friendly_name or uri}", "error")
        speak(f"Failed to open {friendly_name or uri}")
        return False

# -----------------------
# Active window helper (Windows)
# -----------------------
def get_active_window_title():
    """Return the active window title (Windows). Empty string on failure."""
    try:
        user32 = ctypes.windll.user32
        kernel32 = ctypes.windll.kernel32
        h_wnd = user32.GetForegroundWindow()
        length = user32.GetWindowTextLengthW(h_wnd)
        buff = ctypes.create_unicode_buffer(length + 1)
        user32.GetWindowTextW(h_wnd, buff, length + 1)
        return buff.value.lower()
    except Exception:
        return ""

# -----------------------
# Close app helper
# -----------------------
CLOSE_MAP = {
    "chrome": "chrome.exe",
    "google chrome": "chrome.exe",
    "edge": "msedge.exe",
    "firefox": "firefox.exe",
    "brave": "brave.exe",
    "whatsapp": "WhatsApp.exe",
    "whatsapp web": "chrome.exe",
    "youtube": "chrome.exe",
    "spotify": "spotify.exe",
    "vscode": "Code.exe",
    "vs code": "Code.exe",
    "notepad": "notepad.exe",
    "calculator": "Calculator.exe",
    "cmd": "cmd.exe",
    "netflix": "chrome.exe",
    "instagram": "chrome.exe",
    "facebook": "chrome.exe",
    "github": "chrome.exe",
}

def close_app_by_name(name):
    """Try to close app: attempt Alt+F4 if active window matches; else taskkill process."""
    name_l = name.lower().strip()
    title = get_active_window_title()
    # If the active window title contains the app name, send Alt+F4
    if name_l in title or any(k in title for k in [name_l, name_l.replace(" ", "")]):
        try:
            pyautogui.hotkey('alt', 'f4')
            set_hud(f"Closed {name}", "close")
            speak(f"Closed {name}")
            return True
        except Exception:
            pass
    # fallback: taskkill with mapped exe
    proc = CLOSE_MAP.get(name_l)
    if proc:
        try:
            subprocess.run(['taskkill', '/IM', proc, '/F'], check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            set_hud(f"Killed {proc}", "close")
            speak(f"Closed {name}")
            return True
        except Exception:
            set_hud(f"Failed to close {name}", "error")
            speak(f"Failed to close {name}")
            return False
    else:
        # unknown name fallback: attempt taskkill by partial match or Alt+F4
        try:
            pyautogui.hotkey('alt', 'f4')
            set_hud(f"Attempted close {name}", "close")
            speak(f"Attempted to close {name}")
            return True
        except Exception:
            set_hud(f"Close failed {name}", "error")
            speak(f"Close failed for {name}")
            return False

# -----------------------
# Minimal brightness helpers (best-effort)
# -----------------------
def get_brightness_windows():
    try:
        out = subprocess.check_output(['powershell', '-Command', "(Get-WmiObject -Namespace root/WMI -Class WmiMonitorBrightness).CurrentBrightness"], stderr=subprocess.DEVNULL, universal_newlines=True)
        lines = [l.strip() for l in out.splitlines() if l.strip().isdigit()]
        if lines:
            return int(lines[-1])
    except Exception:
        pass
    return None

def set_brightness_windows(value):
    try:
        value = max(0, min(100, int(value)))
        cmd = ['powershell', '-Command', "(Get-WmiObject -Namespace root/WMI -Class WmiMonitorBrightnessMethods).WmiSetBrightness(1,{})".format(value)]
        subprocess.check_output(cmd, stderr=subprocess.DEVNULL)
        set_hud(f"Brightness {value}%", "brightness")
        speak(f"Brightness set to {value} percent")
        return True
    except Exception:
        set_hud("Brightness control not supported", "brightness")
        speak("Brightness control not supported on this PC")
        return False

def change_brightness(delta):
    cur = get_brightness_windows()
    if cur is None:
        set_hud("Bright control unavailable", "brightness")
        speak("Cannot control brightness")
        return False
    new = max(0, min(100, cur + delta))
    return set_brightness_windows(new)

def show_battery_percentage():
    try:
        out = subprocess.check_output(['wmic', 'Path', 'Win32_Battery', 'Get', 'EstimatedChargeRemaining'], stderr=subprocess.DEVNULL, universal_newlines=True)
        lines = [l.strip() for l in out.splitlines() if l.strip().isdigit()]
        if lines:
            pct = lines[-1]
            set_hud(f"Battery: {pct}%", "battery")
            speak(f"Battery is {pct} percent")
            return True
    except Exception:
        pass
    set_hud("Battery info unavailable", "battery")
    speak("Battery information is not available")
    return False

# -----------------------
# Actions (same mapping as before)
# -----------------------
def execute_action(action):
    set_hud(f"{action}", icon=action)
    speak(action.replace("_", " "))

    try:
        # Browser
        if action == "open_whatsapp": return open_url("https://web.whatsapp.com", "WhatsApp Web")
        if action == "open_youtube": return open_url("https://www.youtube.com", "YouTube")
        if action == "open_instagram": return open_url("https://www.instagram.com", "Instagram")
        if action == "open_facebook": return open_url("https://www.facebook.com", "Facebook")
        if action == "open_google": return open_url("https://www.google.com", "Google")
        if action == "open_gmail": return open_url("https://mail.google.com", "Gmail")
        if action == "open_netflix": return open_url("https://www.netflix.com", "Netflix")
        if action == "open_github": return open_url("https://github.com", "GitHub")
        if action == "open_maps": return open_url("https://maps.google.com", "Google Maps")

        # Apps
        if action == "open_calculator": return safe_start_command(["calc"], "Calculator")
        if action == "open_notepad": return safe_start_command(["notepad"], "Notepad")
        if action == "open_cmd": return safe_start_command(["cmd"], "Command Prompt")
        if action == "open_vs_code":
            if shutil.which("code"): return safe_start_command(["code"], "VS Code")
            possible = [r"C:\Users\%USERNAME%\AppData\Local\Programs\Microsoft VS Code\Code.exe", r"C:\Program Files\Microsoft VS Code\Code.exe"]
            for p in possible:
                p = os.path.expandvars(p)
                if os.path.exists(p): return safe_start_command([p], "VS Code")
            set_hud("VS Code not found", "error"); speak("Visual Studio Code not found"); return False
        if action == "open_spotify":
            try:
                return open_windows_uri("spotify:")
            except Exception:
                if shutil.which("spotify"): return safe_start_command(["spotify"], "Spotify")
                set_hud("Spotify not found", "error"); speak("Spotify not found"); return False
        if action == "open_camera": return open_windows_uri("microsoft.windows.camera:", "Camera")
        if action == "open_settings": return open_windows_uri("ms-settings:", "Settings")
        if action == "open_task_manager": return safe_start_command(["taskmgr"], "Task Manager")

        # Media controls
        if action == "play_pause": pyautogui.press("playpause"); set_hud("Play/Pause", "play"); return True
        if action == "next_track": pyautogui.press("nexttrack"); set_hud("Next Track", "next"); return True
        if action == "prev_track": pyautogui.press("prevtrack"); set_hud("Previous Track", "prev"); return True

        # Volume
        if action == "volume_up": pyautogui.press("volumeup"); return True
        if action == "volume_down": pyautogui.press("volumedown"); return True
        if action == "mute": pyautogui.press("volumemute"); return True

        # Brightness
        if action == "brightness_up": return change_brightness(10)
        if action == "brightness_down": return change_brightness(-10)

        # System
        if action == "shutdown": os.system("shutdown /s /t 10"); return True
        if action == "restart": os.system("shutdown /r /t 5"); return True
        if action == "sleep":
            try:
                os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0"); return True
            except Exception:
                set_hud("Sleep failed", "error"); speak("Failed to enter sleep"); return False
        if action == "lock": os.system("rundll32.exe user32.dll,LockWorkStation"); return True
        if action == "show_time":
            import datetime
            now = datetime.datetime.now().strftime("%I:%M %p")
            set_hud(f"Time: {now}", "time"); speak(f"The time is {now}"); return True
        if action == "battery_status": return show_battery_percentage()

    except Exception as e:
        set_hud(f"Error: {e}", "error")
        speak("An error occurred")
        return False

    set_hud("Unknown action", "error"); speak("Unknown action"); return False

# -----------------------
# SEARCH: app-aware typing into current focus
# -----------------------
def perform_search_query(query):
    """Type/search based on active window context."""
    if not query:
        return False
    title = get_active_window_title()
    # prioritize YouTube specific behavior
    if 'youtube' in title:
        # press '/' to focus YouTube search, then type
        try:
            pyautogui.press('/')
            time.sleep(0.2)
            pyautogui.typewrite(query, interval=0.02)
            pyautogui.press('enter')
            set_hud(f"YT Search: {query}", "search")
            speak(f"Searching YouTube for {query}")
            return True
        except Exception:
            pass
    # VS Code / Visual Studio
    if 'visual studio code' in title or 'vs code' in title or 'code -' in title:
        try:
            pyautogui.hotkey('ctrl', 'f')
            time.sleep(0.15)
            pyautogui.typewrite(query, interval=0.02)
            set_hud(f"VSCode Search: {query}", "search")
            speak(f"Searching in VS Code for {query}")
            return True
        except Exception:
            pass
    # Notepad
    if 'notepad' in title:
        try:
            pyautogui.hotkey('ctrl', 'f')
            time.sleep(0.15)
            pyautogui.typewrite(query, interval=0.02)
            pyautogui.press('enter')
            set_hud(f"Notepad Search: {query}", "search")
            speak(f"Searching in notepad for {query}")
            return True
        except Exception:
            pass
    # WhatsApp Desktop or WhatsApp Web
    if 'whatsapp' in title:
        try:
            # WhatsApp desktop: Ctrl+F focuses search; web also works with Ctrl+F
            pyautogui.hotkey('ctrl', 'f')
            time.sleep(0.15)
            pyautogui.typewrite(query, interval=0.02)
            pyautogui.press('enter')
            set_hud(f"WhatsApp Search: {query}", "search")
            speak(f"Searching in WhatsApp for {query}")
            return True
        except Exception:
            pass
    # Browser general fallback: focus address/search bar (Ctrl+L) and search (or type into search box)
    if any(b in title for b in ['chrome', 'edge', 'firefox', 'brave', 'msedge', 'mozilla']):
        try:
            pyautogui.hotkey('ctrl', 'l')
            time.sleep(0.12)
            # use google search in address bar
            url = f"https://www.google.com/search?q={query.replace(' ', '+')}"
            pyautogui.typewrite(url, interval=0.02)
            pyautogui.press('enter')
            set_hud(f"Web Search: {query}", "search")
            speak(f"Searching web for {query}")
            return True
        except Exception:
            pass
    # Generic fallback: just type where cursor is and press Enter
    try:
        pyautogui.typewrite(query, interval=0.02)
        pyautogui.press('enter')
        set_hud(f"Typed: {query}", "type")
        speak(f"Searching for {query}")
        return True
    except Exception:
        set_hud("Search failed", "error")
        speak("Search failed")
        return False

# -----------------------
# Voice listener (extended: search + close commands)
# -----------------------
def voice_listener():
    global voice_typing_enabled
    r = sr.Recognizer()
    try:
        mic = sr.Microphone()
    except Exception:
        set_hud("Microphone not found", icon="mic_error")
        return

    with mic as source:
        r.adjust_for_ambient_noise(source, duration=1)

    set_hud("Voice Ready", "mic")
    speak("Voice ready")

    while True:
        try:
            with mic as source:
                audio = r.listen(source, phrase_time_limit=5)
            try:
                text = r.recognize_google(audio).lower().strip()
            except sr.UnknownValueError:
                continue
            except sr.RequestError:
                set_hud("Speech Service Error", "mic_error")
                time.sleep(2); continue

            set_hud(f"Voice: {text}", "voice")

            # enable/disable voice typing first
            if "disable voice typing" in text:
                voice_typing_enabled = False; set_hud("Voice Typing OFF", "type_off"); speak("Voice typing disabled"); continue
            if "enable voice typing" in text:
                voice_typing_enabled = True; set_hud("Voice Typing ON", "type_on"); speak("Voice typing enabled"); continue

            # If voice typing is on, type everything else (but still allow some commands)
            if voice_typing_enabled:
                # allow "search ..." while in typing mode: treat specially
                if text.startswith("search "):
                    query = text.replace("search", "", 1).strip()
                    perform_search_query(query)
                    continue
                # allow "close ..." in typing mode too
                if text.startswith("close "):
                    name = text.replace("close", "", 1).strip()
                    close_app_by_name(name)
                    continue
                # otherwise type the text
                set_hud("Typing...", "type"); pyautogui.typewrite(text + " ", interval=0.02); continue

            # -------------------------
            # SEARCH (not typing mode)
            # -------------------------
            if text.startswith("search "):
                query = text.replace("search", "", 1).strip()
                if query:
                    perform_search_query(query)
                    continue

            # -------------------------
            # CLOSE APP COMMANDS
            # -------------------------
            if text.startswith("close "):
                name = text.replace("close", "", 1).strip()
                close_app_by_name(name)
                continue

            # -------------------------
            # OTHER COMMANDS (unchanged)
            # -------------------------
            if "open whatsapp" in text: execute_action("open_whatsapp"); continue
            if "open youtube" in text: execute_action("open_youtube"); continue
            if "open instagram" in text: execute_action("open_instagram"); continue
            if "open facebook" in text: execute_action("open_facebook"); continue
            if "open google" in text: execute_action("open_google"); continue
            if "open gmail" in text: execute_action("open_gmail"); continue
            if "open netflix" in text: execute_action("open_netflix"); continue
            if "open github" in text: execute_action("open_github"); continue
            if "open maps" in text or "open google maps" in text: execute_action("open_maps"); continue
            if "open calculator" in text: execute_action("open_calculator"); continue
            if "open notepad" in text: execute_action("open_notepad"); continue
            if "open command prompt" in text or "open cmd" in text: execute_action("open_cmd"); continue
            if "open vs code" in text or "open vscode" in text or "open visual studio code" in text: execute_action("open_vs_code"); continue
            if "open spotify" in text: execute_action("open_spotify"); continue
            if "open camera" in text: execute_action("open_camera"); continue
            if "open settings" in text: execute_action("open_settings"); continue
            if "open task manager" in text or "task manager" in text: execute_action("open_task_manager"); continue
            if "volume up" in text or "increase volume" in text: execute_action("volume_up"); continue
            if "volume down" in text or "decrease volume" in text: execute_action("volume_down"); continue
            if "mute" in text: execute_action("mute"); continue
            if "play" in text or "pause" in text or "play pause" in text: execute_action("play_pause"); continue
            if "next" in text or "next song" in text: execute_action("next_track"); continue
            if "previous" in text or "prev" in text: execute_action("prev_track"); continue
            if "increase brightness" in text or "brightness up" in text: execute_action("brightness_up"); continue
            if "decrease brightness" in text or "brightness down" in text: execute_action("brightness_down"); continue
            if "shutdown" in text: execute_action("shutdown"); continue
            if "restart" in text: execute_action("restart"); continue
            if "sleep" in text or "hibernate" in text: execute_action("sleep"); continue
            if "lock" in text: execute_action("lock"); continue
            if "what time" in text or "tell me the time" in text: execute_action("show_time"); continue
            if "battery" in text or "battery percentage" in text or "show battery" in text: execute_action("battery_status"); continue

        except Exception:
            time.sleep(0.2)

# Start voice thread
voice_thread = threading.Thread(target=voice_listener, daemon=True)
voice_thread.start()

# -----------------------
# HUD Drawer
# -----------------------
def draw_hud(frame):
    if time.time() - hud_timer > HUD_SHOW_TIME:
        return frame
    h, w, _ = frame.shape
    overlay = frame.copy()
    x0, y0 = 10, h - 80
    cv2.rectangle(overlay, (x0, y0), (x0 + 420, y0 + 60), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.55, frame, 0.45, 0, frame)
    if hud_icon:
        cv2.circle(frame, (x0 + 40, y0 + 30), 20, (255, 180, 100), -1)
        cv2.putText(frame, hud_icon[:10], (x0 + 10, y0 + 55),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, (220, 220, 220), 1)
    cv2.putText(frame, hud_text, (x0 + 80, y0 + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    return frame

# -----------------------
# Helpers for gestures
# -----------------------
def distance(p1, p2):
    return int(math.dist(p1, p2))

def fingers_up(hand_landmarks):
    # returns dict of finger name -> bool (True if extended)
    tips = {'thumb':4, 'index':8, 'middle':12, 'ring':16, 'pinky':20}
    res = {}
    # For fingers (except thumb) compare tip y w.r.t pip y
    for f in ['index','middle','ring','pinky']:
        tip = hand_landmarks.landmark[tips[f]]
        pip = hand_landmarks.landmark[tips[f]-2]
        res[f] = tip.y < pip.y  # lower y means higher on image
    # Thumb: compare tip x to ip (handedness not used; we assume mirrored feed)
    thumb_tip = hand_landmarks.landmark[tips['thumb']]
    thumb_ip = hand_landmarks.landmark[3]
    # thumb extended if tip x is to the left of ip (mirrored frame)
    res['thumb'] = thumb_tip.x < thumb_ip.x
    return res

def palm_center(hand_landmarks, frame_w, frame_h):
    # approximate center using wrist(0) and middle_finger_mcp(9)
    w = hand_landmarks.landmark[0]
    m = hand_landmarks.landmark[9]
    cx = int(((w.x + m.x)/2) * frame_w)
    cy = int(((w.y + m.y)/2) * frame_h)
    return (cx, cy)

def time_since(name):
    return time.time() - last_gesture_times.get(name, 0)

def set_gtime(name):
    last_gesture_times[name] = time.time()

# -----------------------
# MAIN HAND GESTURE LOOP (extended)
# -----------------------
print("Starting... Press ESC to exit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands_detector.process(rgb)

    current_time = time.time()
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # coordinates (pixel)
        coords = [(int(lm.x * w), int(lm.y * h)) for lm in hand_landmarks.landmark]
        index = coords[8]
        thumb = coords[4]
        middle = coords[12]
        pinky = coords[20]
        ring = coords[16]

        # move mouse by index finger
        try:
            screen_x = int(hand_landmarks.landmark[8].x * SCREEN_W)
            screen_y = int(hand_landmarks.landmark[8].y * SCREEN_H)
            pyautogui.moveTo(screen_x, screen_y, duration=0.03)
        except Exception:
            pass

        # simple pinch distances
        pinch = distance(index, thumb)
        pinky_touch = distance(pinky, thumb)
        ring_pinch = distance(coords[16], thumb)

        # fingers up
        fup = fingers_up(hand_landmarks)

        # palm center velocity for swipes
        palm = palm_center(hand_landmarks, w, h)
        if prev_palm is None:
            prev_palm = palm
            prev_time = current_time

        dx = palm[0] - prev_palm[0]
        dy = palm[1] - prev_palm[1]
        dt = max(0.001, current_time - prev_time)
        vx = dx / dt
        vy = dy / dt

        # --- Drag Mode (index+thumb pinch & move) ---
        if pinch < 35:
            # start dragging if not already
            if not dragging and time_since("drag") > gesture_cooldown:
                dragging = True
                drag_start = (screen_x, screen_y)
                set_hud("Drag Mode ON", "drag")
                set_gtime("drag")
            # hold left mouse while dragging
            try:
                pyautogui.mouseDown()
            except Exception:
                pass
        else:
            if dragging:
                try:
                    pyautogui.mouseUp()
                except Exception:
                    pass
                dragging = False
                set_hud("Drag Mode OFF", "drag")
                set_gtime("drag_end")

        # --- Left Click / Double Click (quick pinch) ---
        now = current_time
        if pinch < 35 and time_since("pinch_click") > 0.5:
            set_gtime("pinch_click")
        if pinch >= 35 and time_since("pinch_click") < 0.6 and time_since("pinch_release") > 0.5:
            if time_since("pinch_click") < 0.5:
                if now - last_click_time < 0.35:
                    pyautogui.doubleClick(); set_hud("Double Click", "dbl")
                else:
                    pyautogui.click(); set_hud("Left Click", "click")
                last_click_time = now
            set_gtime("pinch_release")

        # --- Right-click hold (pinky+thumb pinch) ---
        if pinky_touch < 40 and time_since("rightclick") > gesture_cooldown:
            pyautogui.rightClick()
            set_hud("Right Click", "rclick")
            set_gtime("rightclick")

        # --- Middle click (ring pinch) ---
        if ring_pinch < 35 and time_since("middleclick") > gesture_cooldown:
            pyautogui.middleClick()
            set_hud("Middle Click", "mclick")
            set_gtime("middleclick")

        # --- Scroll Up / Down via open palm / fist (existing) ---
        if (hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y and
            hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y and
            hand_landmarks.landmark[16].y < hand_landmarks.landmark[14].y and
            hand_landmarks.landmark[20].y < hand_landmarks.landmark[18].y):
            if time_since("scroll") > 0.25:
                pyautogui.scroll(80)
                set_hud("Scroll Up", "up")
                set_gtime("scroll")

        if (hand_landmarks.landmark[8].y > hand_landmarks.landmark[6].y and
            hand_landmarks.landmark[12].y > hand_landmarks.landmark[10].y and
            hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y):
            if time_since("scroll_down") > 0.25:
                pyautogui.scroll(-80)
                set_hud("Scroll Down", "down")
                set_gtime("scroll_down")

        # --- Next / Prev Tab (index+middle up, ring down pattern) existing detection kept ---
        if (hand_landmarks.landmark[8].y < hand_landmarks.landmark[6].y and
            hand_landmarks.landmark[12].y < hand_landmarks.landmark[10].y and
            hand_landmarks.landmark[16].y > hand_landmarks.landmark[14].y):
            if time_since("next_tab") > 0.6:
                pyautogui.hotkey("ctrl", "tab")
                set_hud("Next Tab", "next")
                set_gtime("next_tab")

        if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
            if time_since("prev_tab") > 0.6:
                pyautogui.hotkey("ctrl", "shift", "tab")
                set_hud("Prev Tab", "prev")
                set_gtime("prev_tab")

        # ----------------------------------------------
        # NEW: Gestures based on palm movement (swipes)
        # ----------------------------------------------
        # Swipe Right (fast palm vx > threshold) => Next Track / Forward
        if vx > 1000 and abs(vx) > abs(vy):
            if time_since("swipe_right") > gesture_cooldown:
                execute_action("next_track")
                set_gtime("swipe_right")

        # Swipe Left => Previous Track / Back
        if vx < -1000 and abs(vx) > abs(vy):
            if time_since("swipe_left") > gesture_cooldown:
                execute_action("prev_track")
                set_gtime("swipe_left")

        # Up motion while open palm => volume up (hand move up)
        if vy < -500 and fup['index'] and fup['middle'] and fup['ring'] and fup['pinky']:
            if time_since("palm_up") > gesture_cooldown:
                execute_action("volume_up")
                set_gtime("palm_up")

        # Down motion while open palm => volume down (hand move down)
        if vy > 500 and fup['index'] and fup['middle'] and fup['ring'] and fup['pinky']:
            if time_since("palm_down") > gesture_cooldown:
                execute_action("volume_down")
                set_gtime("palm_down")

        # --- Play/Pause: two-finger tap (index+middle tips quickly touch) ---
        idx_mid_dist = distance(index, middle)
        if idx_mid_dist < 30 and time_since("two_tap") > 0.6:
            execute_action("play_pause")
            set_gtime("two_tap")

        # --- Screenshot: "L" shape detection (index up, thumb away, others down) ---
        if fup['index'] and not fup['middle'] and not fup['ring'] and not fup['pinky'] and fup['thumb'] and time_since("screenshot") > 1.2:
            try:
                pyautogui.hotkey('win', 'shift', 's')
                set_hud("Screenshot", "ss")
                speak("Screenshot captured")
            except Exception:
                set_hud("Screenshot failed", "error")
            set_gtime("screenshot")

        # --- New Tab: quick open palm then close (detect palm open velocity change) ---
        if fup['index'] and fup['middle'] and fup['ring'] and fup['pinky'] and fup['thumb'] and time_since("new_tab") > 1.0:
            if last_gesture_times.get("fist_time") and (current_time - last_gesture_times["fist_time"]) < 0.8:
                pyautogui.hotkey("ctrl", "t")
                set_hud("New Tab", "newtab")
                set_gtime("new_tab")

        # --- Close Tab: fist + lateral shake (shake buffer) ---
        is_fist = (not fup['index'] and not fup['middle'] and not fup['ring'] and not fup['pinky'])
        if is_fist:
            set_gtime("fist_time")
            shake_buffer.append(palm[0])
            if len(shake_buffer) > 8:
                shake_buffer.pop(0)
            if len(shake_buffer) >= 6:
                if (max(shake_buffer) - min(shake_buffer)) > 80 and time_since("close_tab") > 1.2:
                    pyautogui.hotkey("ctrl", "w")
                    set_hud("Close Tab", "ctab")
                    set_gtime("close_tab")
                    shake_buffer = []
        else:
            if len(shake_buffer) > 0 and (current_time - last_gesture_times.get("fist_time", 0)) > 1.0:
                shake_buffer = []

        # --- Go Back / Forward by quick palm swipes (left/right short) ---
        if vx < -1200 and time_since("go_back") > gesture_cooldown:
            pyautogui.hotkey("alt", "left")
            set_hud("Back", "back")
            set_gtime("go_back")
        if vx > 1200 and time_since("go_forward") > gesture_cooldown:
            pyautogui.hotkey("alt", "right")
            set_hud("Forward", "forward")
            set_gtime("go_forward")

        # --- Brightness Up / Down: index pointing up or down (only index up, others down) ---
        if fup['index'] and not fup['middle'] and not fup['ring'] and not fup['pinky']:
            idx_tip_y = hand_landmarks.landmark[8].y
            idx_pip_y = hand_landmarks.landmark[6].y
            if (idx_pip_y - idx_tip_y) > 0.08 and time_since("bright_up") > 1.0:
                execute_action("brightness_up"); set_gtime("bright_up")
            if (idx_tip_y - idx_pip_y) > 0.08 and time_since("bright_down") > 1.0:
                execute_action("brightness_down"); set_gtime("bright_down")

        # --- Mute: full fist (held) ---
        if is_fist and time_since("mute") > 1.5:
            execute_action("mute"); set_gtime("mute")

        # --- Task Manager: five fingers then fist sequence (sequence_buffer) ---
        if fup['index'] and fup['middle'] and fup['ring'] and fup['pinky'] and fup['thumb']:
            sequence_time = current_time
            sequence_buffer.append(("open_all", current_time))
            sequence_buffer = [s for s in sequence_buffer if current_time - s[1] < 1.5]
        if is_fist:
            sequence_buffer.append(("fist", current_time))
            sequence_buffer = [s for s in sequence_buffer if current_time - s[1] < 1.5]
            names = [s[0] for s in sequence_buffer]
            if "open_all" in names and "fist" in names and time_since("taskmgr") > 2.0:
                execute_action("open_task_manager")
                set_gtime("taskmgr")
                sequence_buffer = []

        # --- Lock screen: thumb + ring pinch (thumb + ring close) ---
        thumb_ring_dist = distance(coords[4], coords[16])
        if thumb_ring_dist < 35 and time_since("lock") > 2.0:
            execute_action("lock"); set_gtime("lock")

        # update prev palm
        prev_palm = palm
        prev_time = current_time

    else:
        # no hand detected: clear some transient states
        if dragging:
            try:
                pyautogui.mouseUp()
            except Exception:
                pass
            dragging = False
            set_hud("Drag Mode OFF", "drag")
            set_gtime("drag_end")
        # reset palm so velocities don't spike when hand reappears
        prev_palm = None

    # HUD
    frame = draw_hud(frame) if 'draw_hud' in globals() else frame

    # Show voice typing status
    try:
        cv2.putText(frame, f"Voice Typing: {'ON' if voice_typing_enabled else 'OFF'}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 230, 255), 2)
    except Exception:
        pass

    cv2.imshow("HandVoice Controller HUD (Extended Gestures)", frame)

    # Use ESC to exit
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


Starting... Press ESC to exit
